### Import packages

In [513]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

### Import Dataset

In [520]:
import pandas as pd
import nltk

df = pd.read_excel('Stability_Buzz.xlsx', sheet_name='Relevant')
df = df[['Sentences']]
df['target'] = '0'

data = df
data.columns = ['Sentences', 'target']


data_text = data[['Sentences']]

documents = data_text
documents.head()

,Sentences
0,Baidu formed a strategic partnership with Qual...
1,Blackbaud announced the results of a commissio...
2,Earnings per fully diluted ADS on a GAAP basis...
3,"Fortescue's current director of operations, Gr..."
4,GAAP net loss for the third fiscal quarter was...


In [521]:
data = documents.Sentences.values.tolist()

### Data Preprocess

In [522]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['baidu', 'formed', 'strategic', 'partnership', 'with', 'qualcomm', 'to', 'optimize', 'baidu', 'voice', 'assistant', 'dueros', 'for', 'smartphones', 'on', 'the', 'qualcomm', 'snapdragon', 'tm', 'mobile', 'platform', 'including', 'the', 'upcoming', 'snapdragon', 'mobile', 'platform']]


In [523]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])



/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['baidu', 'formed', 'strategic', 'partnership', 'with', 'qualcomm', 'to', 'optimize', 'baidu', 'voice', 'assistant', 'dueros', 'for', 'smartphones', 'on', 'the', 'qualcomm', 'snapdragon', 'tm', 'mobile', 'platform', 'including', 'the', 'upcoming', 'snapdragon', 'mobile', 'platform']


In [524]:
from gensim.utils import simple_preprocess
from nltk import word_tokenize
stop_words = stopwords.words('english')
stop_words.extend(['included','including','includes','one','two','three','general','generally','way','better','need','really','believe','provide','say','saying','says','include','said','also','would','could','end','recent','recently','us','like','make','may','look','still','come','see','across','even','much','get','allow','allows','allowed','allowing','well','go','gone','going','that','thats','take','thing','come','move','give','help','put','dont','people','got','kind','know','day','think','continue','look','year','years','new','use','there','want','wanted','due','become','organ'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


 

In [493]:
def clean_text(texts):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in stop_words and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 

tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(texts))

In [525]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['baidu', 'form', 'strategic', 'partnership', 'qualcomm', 'optimize', 'baidu', 'voice', 'assistant', 'duero', 'smartphone', 'qualcomm', 'snapdragon', 'mobile', 'platform', 'upcome', 'snapdragon', 'mobile', 'platform']]


### Create Dictionary

In [526]:
texts = data_lemmatized
id2word = gensim.corpora.Dictionary(texts)


# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1)]]


In [527]:
id2word[1]

'baidu'

In [528]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('assistant', 1),
  ('baidu', 2),
  ('duero', 1),
  ('form', 1),
  ('mobile', 2),
  ('optimize', 1),
  ('partnership', 1),
  ('platform', 2),
  ('qualcomm', 2),
  ('smartphone', 1),
  ('snapdragon', 2),
  ('strategic', 1),
  ('upcome', 1),
  ('voice', 1)]]

### Build LDA model

In [529]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True) 

In [530]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"growth" + 0.014*"sale" + 0.012*"quarter" + 0.011*"increase" + '
  '0.011*"revenue" + 0.010*"market" + 0.009*"business" + 0.008*"strong" + '
  '0.007*"continue" + 0.006*"product"'),
 (1,
  '0.008*"market" + 0.007*"product" + 0.007*"company" + 0.007*"sale" + '
  '0.005*"customer" + 0.005*"revenue" + 0.004*"low" + 0.004*"president" + '
  '0.004*"solution" + 0.004*"global"'),
 (2,
  '0.011*"company" + 0.007*"revenue" + 0.006*"customer" + 0.005*"business" + '
  '0.005*"report" + 0.004*"investment" + 0.004*"plan" + 0.004*"first" + '
  '0.004*"growth" + 0.004*"ceo"'),
 (3,
  '0.009*"company" + 0.008*"growth" + 0.007*"customer" + 0.005*"management" + '
  '0.005*"business" + 0.005*"service" + 0.005*"grow" + 0.005*"application" + '
  '0.005*"market" + 0.004*"result"'),
 (4,
  '0.014*"company" + 0.013*"growth" + 0.008*"revenue" + 0.006*"expect" + '
  '0.005*"market" + 0.005*"rate" + 0.004*"business" + 0.004*"base" + '
  '0.004*"technology" + 0.003*"make"')]


####  Visualize the topics

In [534]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.080947 -0.085355       1        1  38.800934
1     -0.001372  0.061951       2        1  19.033991
4      0.102371 -0.055274       3        1  16.182228
2     -0.029690  0.048924       4        1  15.887640
3      0.009638  0.029754       5        1  10.095201, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
144   Default  339.000000           growth  339.000000  30.0000  30.0000
135   Default  171.000000          company  171.000000  29.0000  29.0000
308   Default   76.000000         customer   76.000000  28.0000  28.0000
125   Default  147.000000             sale  147.000000  27.0000  27.0000
227   Default   30.000000         solution   30.000000  26.0000  26.0000
46    Default  121.000000          quarter  121.000000  25.0000  25.0000
754   Default   35.000000       management   35.000000  24.0000  24.0000
64    Default   40.000000           fiscal   40.000000  23.0000  23.0000
1429  Default   20.000000      application   20.000000  22.0000  22.0000
382   Default   39.000000             rate   39.000000  21.0000  21.0000
169   Default   73.000000         continue   73.000000  20.0000  20.0000
65    Default  114.000000         increase  114.000000  19.0000  19.0000
146   Default   93.000000          product   93.000000  18.0000  18.0000
651   Default   20.000000            order   20.000000  17.0000  17.0000
1979  Default   12.000000  episurf_medical   12.000000  16.0000  16.0000
174   Default   27.000000          organic   27.000000  15.0000  15.0000
66    Default   30.000000              net   30.000000  14.0000  14.0000
27    Default   46.000000       investment   46.000000  13.0000  13.0000
41    Default   66.000000            first   66.000000  12.0000  12.0000
252   Default   44.000000           report   44.000000  11.0000  11.0000
1538  Default   27.000000  chief_executive   27.000000  10.0000  10.0000
428   Default   39.000000          improve   39.000000   9.0000   9.0000
401   Default   34.000000             make   34.000000   8.0000   8.0000
151   Default   64.000000          service   64.000000   7.0000   7.0000
735   Default    7.000000             list    7.000000   6.0000   6.0000
469   Default   33.000000       production   33.000000   5.0000   5.0000
159   Default   44.000000            group   44.000000   4.0000   4.0000
89    Default   50.000000             plan   50.000000   3.0000   3.0000
273   Default   34.000000        president   34.000000   2.0000   2.0000
905   Default   28.000000            value   28.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
3342   Topic5    2.815637        recognize    6.053649   1.5276  -6.5978
66     Topic5    6.977946              net   30.419136   0.8208  -5.6903
135    Topic5   17.771988          company  171.003601   0.0290  -4.7554
329    Topic5    4.569592           income   15.502522   1.0715  -6.1136
82     Topic5    4.726899           enable   16.536663   1.0408  -6.0797
227    Topic5    6.674942         solution   30.709314   0.7669  -5.7347
151    Topic5   10.026196          service   64.275322   0.4351  -5.3278
577    Topic5    8.195358         strategy   46.830437   0.5501  -5.5295
460    Topic5    5.933833          support   27.155367   0.7722  -5.8523
30     Topic5    9.190822           result   64.246811   0.3486  -5.4148
564    Topic5    3.829806             long   11.720320   1.1746  -6.2902
381    Topic5    9.862916             grow   90.651253   0.0749  -5.3442
159    Topic5    6.648141            group   44.095909   0.4011  -5.7387
144    Topic5   15.978679           growth  339.420410  -0.7629  -4.8618
415    Topic5   10.436445         business  126.981201  -0.2056  -5.2877
905    Topic5    5.491140            value   28.530607   0.6453  -5.9299
27

### Test New Unseen Document

In [478]:
from nltk import word_tokenize
text = 'And a quick one as well on your working capital. I was surprised to see an upward movement in the fourth quarter, which is a bit counter-intuitive given the steady rise in oil prices. Could you elaborate a little bit on this?'
bow = id2word.doc2bow(clean_text(text))
print(lda_model[bow])


[(1, 0.54317534), (2, 0.42331395), (4, 0.018748444)]
